<a href="https://colab.research.google.com/github/happyhillll/work/blob/master/Kurly%20Text%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 준비: 네이버 IP 불러와서 크롤링


In [1]:
import os
import sys
import urllib.request
import datetime
import time
import json

client_id = "eK7_u9Zvew3AU7jIsuqm"
client_secret = "l_tBfpZw1k"

In [2]:
# [CODE 1]
def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)

    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [3]:
# [CODE 2]
def getNaverSearch(node, srcText, start, display):
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display)

    url = base + node + parameters
    responseDecode = getRequestUrl(url)  # [CODE 1]

    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)

In [4]:
# [CODE 3]
def getPostData(post, jsonResult, cnt):
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']

    pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')

    jsonResult.append({'cnt': cnt, 'title': title, 'description': description,
                       'org_link': org_link, 'link': org_link, 'pDate': pDate})
    return

In [5]:
# [CODE 0]
def main():
    node = 'news'  # 크롤링 할 대상
    srcText = input('검색어를 입력하세요: ')
    cnt = 0
    jsonResult = []

    jsonResponse = getNaverSearch(node, srcText, 1, 100)  # [CODE 2]
    total = jsonResponse['total']

    while ((jsonResponse != None) and (jsonResponse['display'] != 0)):
        for post in jsonResponse['items']:
            cnt += 1
            getPostData(post, jsonResult, cnt)  # [CODE 3]

        start = jsonResponse['start'] + jsonResponse['display']
        jsonResponse = getNaverSearch(node, srcText, start, 100)  # [CODE 2]

    print('전체 검색 : %d 건' % total)

    with open('%s_naver_%s.json' % (srcText, node), 'w', encoding='utf8') as outfile:
        jsonFile = json.dumps(jsonResult, indent=4, sort_keys=True, ensure_ascii=False)

        outfile.write(jsonFile)

    print("가져온 데이터 : %d 건" % (cnt))
    print('%s_naver_%s.json SAVED' % (srcText, node))


if __name__ == '__main__':
    main()

검색어를 입력하세요: 마켓컬리가 상장철회하다
[2023-01-19 05:23:36.235736] Url Request Success
[2023-01-19 05:23:37.247088] Url Request Success
[2023-01-19 05:23:38.242067] Url Request Success
[2023-01-19 05:23:39.230234] Url Request Success
[2023-01-19 05:23:40.228308] Url Request Success
[2023-01-19 05:23:41.220323] Url Request Success
[2023-01-19 05:23:42.193523] Url Request Success
[2023-01-19 05:23:43.185233] Url Request Success
[2023-01-19 05:23:44.195960] Url Request Success
[2023-01-19 05:23:45.132948] Url Request Success
[2023-01-19 05:23:45.734226] Url Request Success
전체 검색 : 938 건
가져온 데이터 : 938 건
마켓컬리가 상장철회하다_naver_news.json SAVED


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
cd /content/ 마켓컬리가 상장철회하다_naver_news.json

[Errno 2] No such file or directory: '/content/ 마켓컬리가 상장철회하다_naver_news.json'
/content


In [8]:
from glob import glob
for filename in glob('*.json'):
  print(filename)

마켓컬리가 상장철회하다_naver_news.json


In [10]:
import pandas as pd

In [11]:
# 각 파일의 shape, head 출력
from IPython.display import display

def preview():
  for filename in glob('*.json'):
    df = pd.read_json(filename)
    print(filename, df.shape)
    display(df.head())
    print('\n')
    
preview()

마켓컬리가 상장철회하다_naver_news.json (938, 6)


,cnt,description,link,org_link,pDate,title
0,1,이커머스 업계의 대어급 IPO로 주목받았던 경쟁업체 <b>마켓컬리가</b> 올해 초...,https://weekly.cnbnews.com/news/article.html?n...,https://weekly.cnbnews.com/news/article.html?n...,2023-01-19 11:50:00,"[2023 IPO] 새벽배송 유일 흑자 오아시스<b>마켓</b>, 올해 이커머스 1..."
1,2,우선 최근 IPO를 준비 중이던 기업들이 줄줄이 <b>상장</b>을 미루거나 <b>...,http://www.kukinews.com/newsView/kuk202301190007,http://www.kukinews.com/newsView/kuk202301190007,2023-01-19 07:04:00,SK에코플랜트 IPO 딜레마…재무 부담에 고평가 지적도
2,3,특히 최근에 시장의 큰 기대를 모았던 <b>마켓컬리</b> 운영사인 <b>컬리</b...,http://news.mt.co.kr/mtview.php?no=20230118135...,http://news.mt.co.kr/mtview.php?no=20230118135...,2023-01-19 06:00:00,"IPO 시장 암울…SK에코플랜트·현대엔지니어링, 고민 커진다"
3,4,"있던 <b>컬리</b>, 현대삼호중공업, 현대엔지니어링, 원스토어, 밀리의서재, S...",https://byline.network/?p=9004111222493303,https://byline.network/?p=9004111222493303,2023-01-18 21:52:00,[진단] 인터넷은행은 정말로 금융 산업을 혁신했나
4,5,‘새벽배송’으로 유명한 <b>마켓컬리</b> 운영사 ‘<b>컬리</b>’가 결국 코...,https://www.mk.co.kr/article/10605476,https://www.mk.co.kr/article/10605476,2023-01-18 21:18:00,글로벌 경제 악화에 <b>상장</b> 포기했지만… 非식품 강화하고 물류 투자 나선 ...


In [12]:
df=pd.read_json('마켓컬리가 상장철회하다_naver_news.json')

In [13]:
df

,cnt,description,link,org_link,pDate,title
0,1,이커머스 업계의 대어급 IPO로 주목받았던 경쟁업체 <b>마켓컬리가</b> 올해 초...,https://weekly.cnbnews.com/news/article.html?n...,https://weekly.cnbnews.com/news/article.html?n...,2023-01-19 11:50:00,"[2023 IPO] 새벽배송 유일 흑자 오아시스<b>마켓</b>, 올해 이커머스 1..."
1,2,우선 최근 IPO를 준비 중이던 기업들이 줄줄이 <b>상장</b>을 미루거나 <b>...,http://www.kukinews.com/newsView/kuk202301190007,http://www.kukinews.com/newsView/kuk202301190007,2023-01-19 07:04:00,SK에코플랜트 IPO 딜레마…재무 부담에 고평가 지적도
2,3,특히 최근에 시장의 큰 기대를 모았던 <b>마켓컬리</b> 운영사인 <b>컬리</b...,http://news.mt.co.kr/mtview.php?no=20230118135...,http://news.mt.co.kr/mtview.php?no=20230118135...,2023-01-19 06:00:00,"IPO 시장 암울…SK에코플랜트·현대엔지니어링, 고민 커진다"
3,4,"있던 <b>컬리</b>, 현대삼호중공업, 현대엔지니어링, 원스토어, 밀리의서재, S...",https://byline.network/?p=9004111222493303,https://byline.network/?p=9004111222493303,2023-01-18 21:52:00,[진단] 인터넷은행은 정말로 금융 산업을 혁신했나
4,5,‘새벽배송’으로 유명한 <b>마켓컬리</b> 운영사 ‘<b>컬리</b>’가 결국 코...,https://www.mk.co.kr/article/10605476,https://www.mk.co.kr/article/10605476,2023-01-18 21:18:00,글로벌 경제 악화에 <b>상장</b> 포기했지만… 非식품 강화하고 물류 투자 나선 ...
...,...,...,...,...,...,...
933,934,1년 전 <b>상장</b>을 추진하다가 <b>철회</b>한 카카오게임즈의 지난해 매...,https://www.hankyung.com/article/2019051918721,https://www.hankyung.com/article/2019051918721,2019-05-19 18:31:00,기업공개 시장 &apos;서머 랠리&apos; 온다
934,935,이마트·롯데마트·GS리테일·쿠팡·<b>마켓컬리</b> 등 대부분의 유통업계는 물류 ...,http://magazine.hankyung.com/business/apps/new...,http://magazine.hankyung.com/business/apps/new...,2019-04-23 10:16:00,유통 공룡 혁신 나서는데 온라인 전략 약한 홈플러스
935,936,하지만 리츠 <b>상장 철회</b>는 이런 임 사장의 플랜에 직격탄을 날렸다. 홈플...,http://www.fntimes.com/html/view.php?ud=201904...,http://www.fntimes.com/html/view.php?ud=201904...,2019-04-15 00:02:00,"위기의 대형마트(3·끝) 홈플러스, 리츠 무산 여파 이커머스 사업 불투명"
936,937,단기간에 차익을 실현할 수 있었지만 지난해 12월 SNK <b>상장 철회</b>로 ...,http://www.thebell.co.kr/front/free/contents/n...,http://www.thebell.co.kr/front/free/contents/n...,2019-04-04 08:48:00,"[더벨]&apos;시험대&apos; 오른 알펜루트자산운용, 리스크관리 &apos;경..."


In [14]:
description=df['description']

In [15]:
print(description)

0      이커머스 업계의 대어급 IPO로 주목받았던 경쟁업체 <b>마켓컬리가</b> 올해 초...
1      우선 최근 IPO를 준비 중이던 기업들이 줄줄이 <b>상장</b>을 미루거나 <b>...
2      특히 최근에 시장의 큰 기대를 모았던 <b>마켓컬리</b> 운영사인 <b>컬리</b...
3      있던 <b>컬리</b>, 현대삼호중공업, 현대엔지니어링, 원스토어, 밀리의서재, S...
4      ‘새벽배송’으로 유명한 <b>마켓컬리</b> 운영사 ‘<b>컬리</b>’가 결국 코...
                             ...                        
933    1년 전 <b>상장</b>을 추진하다가 <b>철회</b>한 카카오게임즈의 지난해 매...
934    이마트·롯데마트·GS리테일·쿠팡·<b>마켓컬리</b> 등 대부분의 유통업계는 물류 ...
935    하지만 리츠 <b>상장 철회</b>는 이런 임 사장의 플랜에 직격탄을 날렸다. 홈플...
936    단기간에 차익을 실현할 수 있었지만 지난해 12월 SNK <b>상장 철회</b>로 ...
937    식품배송 서비스 ‘<b>마켓컬리</b>’로 알려진 <b>컬리</b>(옛 더파마스)의...
Name: description, Length: 938, dtype: object


In [16]:
description.to_csv('description.csv')

# Description 파일 확인하기

In [17]:
description.tail()

933    1년 전 <b>상장</b>을 추진하다가 <b>철회</b>한 카카오게임즈의 지난해 매...
934    이마트·롯데마트·GS리테일·쿠팡·<b>마켓컬리</b> 등 대부분의 유통업계는 물류 ...
935    하지만 리츠 <b>상장 철회</b>는 이런 임 사장의 플랜에 직격탄을 날렸다. 홈플...
936    단기간에 차익을 실현할 수 있었지만 지난해 12월 SNK <b>상장 철회</b>로 ...
937    식품배송 서비스 ‘<b>마켓컬리</b>’로 알려진 <b>컬리</b>(옛 더파마스)의...
Name: description, dtype: object

In [18]:
description.iloc[:3]

0    이커머스 업계의 대어급 IPO로 주목받았던 경쟁업체 <b>마켓컬리가</b> 올해 초...
1    우선 최근 IPO를 준비 중이던 기업들이 줄줄이 <b>상장</b>을 미루거나 <b>...
2    특히 최근에 시장의 큰 기대를 모았던 <b>마켓컬리</b> 운영사인 <b>컬리</b...
Name: description, dtype: object

## 데이터 전처리

1.   html 태그 삭제



In [19]:
import re

to_clean = re.compile('<.*?>')
cleantext=re.sub(to_clean,'',str(description))

print(cleantext)

0      이커머스 업계의 대어급 IPO로 주목받았던 경쟁업체 마켓컬리가 올해 초...
1      우선 최근 IPO를 준비 중이던 기업들이 줄줄이 상장을 미루거나 ...
2      특히 최근에 시장의 큰 기대를 모았던 마켓컬리 운영사인 컬리</b...
3      있던 컬리, 현대삼호중공업, 현대엔지니어링, 원스토어, 밀리의서재, S...
4      ‘새벽배송’으로 유명한 마켓컬리 운영사 ‘컬리’가 결국 코...
                             ...                        
933    1년 전 상장을 추진하다가 철회한 카카오게임즈의 지난해 매...
934    이마트·롯데마트·GS리테일·쿠팡·마켓컬리 등 대부분의 유통업계는 물류 ...
935    하지만 리츠 상장 철회는 이런 임 사장의 플랜에 직격탄을 날렸다. 홈플...
936    단기간에 차익을 실현할 수 있었지만 지난해 12월 SNK 상장 철회로 ...
937    식품배송 서비스 ‘마켓컬리’로 알려진 컬리(옛 더파마스)의...
Name: description, Length: 938, dtype: object


In [20]:
type(cleantext)

str

In [21]:
with open("cleantext.csv","w") as file:
    file.write(cleantext + "\n")

2. 문장분리

In [23]:
import pandas as pd
import nltk
import konlpy
import kss
from tqdm.notebook import tqdm

ModuleNotFoundError: ignored

In [ ]:
data=pd.read_csv('/content/description.csv',encoding='utf-8')

In [ ]:

from nltk.tokenize import sent_tokenize
import pandas as pd
import nltk
nltk.download('punkt')

sentence_tokenized_text = []
data = pd.read_csv('/content/description.csv', encoding = 'utf-8')
for article in tqdm(cleantext):
  temp = sent_tokenize(article)
  for sent in temp:
    sentence_tokenized_text.append(sent)
sentence_tokenized_text
     

## html 삭제, space 삭제

In [24]:
#html 삭제 ,extra space 삭제, lowercasing 
import re

def clean_text(texts):
    corpus = []
    for i in range(0, len(texts)):
        review = re.sub(r'[@%\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~$\'\"]', '',str(texts[i])) #remove punctuation
        review = review.lower() #lower case
        review = re.sub(r'\s+', ' ', review) #remove extra space
        review = re.sub(r'<[^>]+>','',review) #remove Html tags
        review = re.sub(r'\s+', ' ', review) #remove spaces
        review = re.sub(r"^\s+", '', review) #remove space from start
        review = re.sub(r'\s+$', '', review) #remove space from the end
        corpus.append(review)
    return corpus

In [25]:
#실행
basic_preprocessed_corpus = clean_text(description)

In [26]:
#확인
for i in range(0, 10):
    print(basic_preprocessed_corpus[i])

이커머스 업계의 대어급 ipo로 주목받았던 경쟁업체 마켓컬리가 올해 초 상장을 철회한 것과 상반된 행보다 공모 계획도 오는 2월 중 상장을 완료를 목표로 하는 등 속도전을 예고했다 계획대로 상장이 추진될 경우
우선 최근 ipo를 준비 중이던 기업들이 줄줄이 상장을 미루거나 철회했다 고금리로 인한 자금조달 위축 시장 침체가 영향을 미쳤다 새벽배송 서비스 마켓컬리를 운영하는 컬리는 올해 초 코스피 상장 추진을
특히 최근에 시장의 큰 기대를 모았던 마켓컬리 운영사인 컬리마저 투자심리위축을 이유로 상장을 철회하면서 ipo 시장이 급격히 위축되는 분위기다 관련 업계에서는 현대엔지니어링이 상장을 철회했기 때문에 주관사
있던 컬리 현대삼호중공업 현대엔지니어링 원스토어 밀리의서재 sk쉴더스 등 대어들이 상장을 철회하면 현재까지 당근마켓 오아시스 번개장터 nh투자증권 등 다양한 기업들과 손을 잡았다 케이뱅크가 비금융
‘새벽배송’으로 유명한 마켓컬리 운영사 ‘컬리’가 결국 코스피 상장을 연기하기로 했다 기업공개ipo 국내 증시를 포기하고 미국 나스닥에 상장할 것이라는 관측이 나오는가 하면 아예 상장을 철회할
이를 반영하듯 연초부터 ipo 준비 기업들의 상장 연기 및 철회 소식이 이어졌다 이 중 시장에서 가장 크게 관심을 받은 이슈는 단연 컬리의 상장 추진 연기 소식이다 마켓컬리 운영사인 컬리는 이커머스 상장을
컬리와 한국조선해양자회사인 현대삼호중공업은 이달 초 투자 심리 위축을 고려해 상장을 철회했다 케이뱅크와 골프존카운티도 상반기 중 ipo가 불투명한 상황이다 새벽배송 전문업체 오아시스마켓은 최근
여기에 올해 초 상장할 것으로 기대를 모았던 현대삼호중공업과 마켓컬리가 줄줄이 상장을 철회했다 반면 기업인수목적회사인 스팩은 같은 시기 증권사와 비상장사들로부터 주목을 받기 시작했다 스팩 합병의
새해가 밝자마자 컬리는 지난해부터 추진해온 상장을 철회한다고 밝혔다 컬리는 보도자료를 통해 글로벌 필요한 컬리 입장에서 물류센터가 늘어나는 것이 긍정적인 효과만 낼 수 있을지 의문이라고 말했

In [30]:
#띄어쓰기 검사
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-yv2ke9nu
  Running command git clone --filter=blob:none --quiet https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-yv2ke9nu
  Resolved https://github.com/haven-jeon/PyKoSpacing.git to commit 1f8d11c59ac93525432f164e4c237ece4e298691
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.8/511.8 MB 2.7 MB/s eta 0:00:00
  Created wheel for pykospacing: filename=pykospacing-0.5-py3-none-any.whl size=2268637 sha256=0ceab71a0f9a5ddbc48ee17d0e4e685bd480b49c67a307c89e5558e241b6adef
  Stored in directory: /tmp/pip-ephem-wheel-cache-cca44et2/wheels/79/a0/33/16f2cd03d21f76a663f5d69a0b96f0351335385349136fbd03
Successfully built pykospacing
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.9.2
    Uninstalling tensorflow-2.9.2:
      Successfully

In [31]:

#실행
from pykospacing import Spacing
spacing=Spacing()
lines_space=(basic_preprocessed_corpus)
lines_space

['이커머스 업계의 대어급 ipo로 주목받았던 경쟁업체 마켓컬리가 올해 초 상장을 철회한 것과 상반된 행보다 공모 계획도 오는 2월 중 상장을 완료를 목표로 하는 등 속도전을 예고했다 계획대로 상장이 추진될 경우',
 '우선 최근 ipo를 준비 중이던 기업들이 줄줄이 상장을 미루거나 철회했다 고금리로 인한 자금조달 위축 시장 침체가 영향을 미쳤다 새벽배송 서비스 마켓컬리를 운영하는 컬리는 올해 초 코스피 상장 추진을',
 '특히 최근에 시장의 큰 기대를 모았던 마켓컬리 운영사인 컬리마저 투자심리위축을 이유로 상장을 철회하면서 ipo 시장이 급격히 위축되는 분위기다 관련 업계에서는 현대엔지니어링이 상장을 철회했기 때문에 주관사',
 '있던 컬리 현대삼호중공업 현대엔지니어링 원스토어 밀리의서재 sk쉴더스 등 대어들이 상장을 철회하면 현재까지 당근마켓 오아시스 번개장터 nh투자증권 등 다양한 기업들과 손을 잡았다 케이뱅크가 비금융',
 '‘새벽배송’으로 유명한 마켓컬리 운영사 ‘컬리’가 결국 코스피 상장을 연기하기로 했다 기업공개ipo 국내 증시를 포기하고 미국 나스닥에 상장할 것이라는 관측이 나오는가 하면 아예 상장을 철회할',
 '이를 반영하듯 연초부터 ipo 준비 기업들의 상장 연기 및 철회 소식이 이어졌다 이 중 시장에서 가장 크게 관심을 받은 이슈는 단연 컬리의 상장 추진 연기 소식이다 마켓컬리 운영사인 컬리는 이커머스 상장을',
 '컬리와 한국조선해양자회사인 현대삼호중공업은 이달 초 투자 심리 위축을 고려해 상장을 철회했다 케이뱅크와 골프존카운티도 상반기 중 ipo가 불투명한 상황이다 새벽배송 전문업체 오아시스마켓은 최근',
 '여기에 올해 초 상장할 것으로 기대를 모았던 현대삼호중공업과 마켓컬리가 줄줄이 상장을 철회했다 반면 기업인수목적회사인 스팩은 같은 시기 증권사와 비상장사들로부터 주목을 받기 시작했다 스팩 합병의',
 '새해가 밝자마자 컬리는 지난해부터 추진해온 상장을 철회한다고 밝혔다 컬리는 보도자료를 통해 글로벌 필요한 컬리 입장에서 물류센터가

In [32]:
final_output = pd.DataFrame(lines_space)
len(final_output)

938

In [33]:

#데이터 새로 쓰기
final_output.to_csv('Kurlycleaned_.csv',encoding = 'utf-8-sig')

# TF  -  IDF 를 이용한 단어 추출

In [34]:
data=pd.read_csv('/content/Kurlycleaned_.csv',encoding='utf-8')

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]

tfidfv = TfidfVectorizer().fit(data['0'])
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
{'이커머스': 4395, '업계의': 3713, '대어급': 1764, 'ipo로': 502, '주목받았던': 4916, '경쟁업체': 934, '마켓컬리가': 2080, '올해': 4029, '상장을': 2930, '철회한': 5266, '것과': 846, '상반된': 2882, '행보다': 5962, '공모': 1038, '계획도': 964, '오는': 3948, '2월': 199, '완료를': 4040, '목표로': 2231, '하는': 5840, '속도전을': 3172, '예고했다': 3888, '계획대로': 963, '상장이': 2933, '추진될': 5382, '경우': 921, '우선': 4097, '최근': 5345, 'ipo를': 503, '준비': 4951, '중이던': 4996, '기업들이': 1341, '줄줄이': 4973, '미루거나': 2325, '철회했다': 5280, '고금리로': 981, '인한': 4438, '자금조달': 4542, '위축': 4151, '시장': 3324, '침체가': 5455, '영향을': 3880, '미쳤다': 2345, '새벽배송': 3002, '서비스': 3038, '마켓컬리를': 2087, '운영하는': 4111, '컬리는': 5507, '코스피': 5555, '상장': 2886, '추진을': 5384, '특히': 5703, '최근에': 5347, '시장의': 3339, '기대를': 1277, '모았던': 2217, '마켓컬리': 2077, '운영사인': 4108, '컬리마저': 5514, '투자심리위축을': 5663, '이유로': 4381, '철회하면서': 5261, 'ipo': 491, '시장이': 3340, '급격히': 1229, '위축되는': 4155, '분위기다': 2676, '관련': 1097, '업계에서는': 3710, '현대엔지니어링이': 5988, '

In [51]:
dic=tfidfv.vocabulary_.items()
print(dic)

dict_items([('이커머스', 4395), ('업계의', 3713), ('대어급', 1764), ('ipo로', 502), ('주목받았던', 4916), ('경쟁업체', 934), ('마켓컬리가', 2080), ('올해', 4029), ('상장을', 2930), ('철회한', 5266), ('것과', 846), ('상반된', 2882), ('행보다', 5962), ('공모', 1038), ('계획도', 964), ('오는', 3948), ('2월', 199), ('완료를', 4040), ('목표로', 2231), ('하는', 5840), ('속도전을', 3172), ('예고했다', 3888), ('계획대로', 963), ('상장이', 2933), ('추진될', 5382), ('경우', 921), ('우선', 4097), ('최근', 5345), ('ipo를', 503), ('준비', 4951), ('중이던', 4996), ('기업들이', 1341), ('줄줄이', 4973), ('미루거나', 2325), ('철회했다', 5280), ('고금리로', 981), ('인한', 4438), ('자금조달', 4542), ('위축', 4151), ('시장', 3324), ('침체가', 5455), ('영향을', 3880), ('미쳤다', 2345), ('새벽배송', 3002), ('서비스', 3038), ('마켓컬리를', 2087), ('운영하는', 4111), ('컬리는', 5507), ('코스피', 5555), ('상장', 2886), ('추진을', 5384), ('특히', 5703), ('최근에', 5347), ('시장의', 3339), ('기대를', 1277), ('모았던', 2217), ('마켓컬리', 2077), ('운영사인', 4108), ('컬리마저', 5514), ('투자심리위축을', 5663), ('이유로', 4381), ('철회하면서', 5261), ('ipo', 491), ('시장이', 3340), ('급격히', 1229), ('위축되는', 

In [56]:
df=pd.DataFrame(dic,columns=['word','score'])
df

,word,score
0,이커머스,4395
1,업계의,3713
2,대어급,1764
3,ipo로,502
4,주목받았던,4916
...,...,...
6131,여부를,3771
6132,2000억원,127
6133,베팅했던,2534
6134,카카오가,5469


In [58]:
#빈도 내림차순으로 정리
dff =df.sort_values(by='score', ascending=False).reset_index(drop=True)
dff

,word,score
0,힘입어,6135
1,힘들어지는,6134
2,힘들다는,6133
3,힘들,6132
4,희박해지는,6131
...,...,...
6131,08월,4
6132,07일,3
6133,02일,2
6134,01월,1


In [59]:
dff.to_csv('Kurly_tfidf.csv')

# 응집도 기반 키워드 추출

In [36]:


import pandas as pd
import numpy as np

class Word:
#단어 리스트
#key: 단어 길이(리뷰에서 공백을 기준으로 분리된 토큰)
#word: 실제 단어
#cnt: key와 동일한 단어의 출현 빈도수
#freq: key가 포함된 (key로 시작하는) 단어의 출현 빈도수
    def __init__(self, word):
        self.key = len(word)
        self.word = word
        self.cnt = 1
        self.freq = 1

class Node:
#id: 노드 아이디
#pid: 부모 노드의 아이디
#dep: 트리에서 깊이
#w : word 클래스 연결
#coh: 응집도 점수
    def __init__(self, id, word):
        self.id = id
        self.pid = -1
        self.dep = 1
        self.w = word
        self.coh = 0

#word 클래스 sort 해야해서 heap으로 함!
#heap_sort 사용 예정
class Heap: 
    def __init__(self, L = []):
        self.A = L     
        self.make_heap()
    
    def __str__(self):
        return str(self.A)
    def __len__(self):
        return len(self.A)
    
    def heapify_down(self,k,n):
        while 2*k+1 < n :
            L, R = 2*k+1, 2*k+2
            if L < n and self.A[L].key > self.A[k].key : m = L
            else: m = k
            if R < n and self.A[R].key > self.A[m].key : m = R
            #m = A[k],A[L],A[K] 중 최대의 인덱스

            if m != k: #A[k] 가 최소가 아니라면 힙 성질 위배
                self.A[k], self.A[m] = self.A[m],self.A[k]
                k = m 
            else:
                break
    
    def make_heap(self):
        n = len(self.A)
        for k in range(n-1,-1,-1):
            self.heapify_down(k,n)

    def heap_sort(self):
        n = len(self.A)
        for k in range(len(self.A)-1, -1, -1):
            self.A[0], self.A[k] = self.A[k], self.A[0]
            n = n-1
            self.heapify_down(0,n)
    def copy(self):
        return self.A


def binarySearch(list, item, left,right):
    if left > right:
        return False
    else:
        mid = (left + right)//2
        if item == A[mid].w.word:
            return True
        elif item < A[mid].w.key:
            return binarySearch(A,item,left,mid-1)
        else:
            return binarySearch(A,item,mid+1,right)

In [37]:
#응집도 분석할 단어 리스트 추출 및 초기화

def getWordList(wdset):
    wordlist = []
    for i in range(len(wdset)):
        if len(wdset[i]) < 1:
            continue
        found = 0
        for a in range(len(wordlist)):
            if wdset[i] == wordlist[a].word:
                wordlist[a].cnt += 1
                wordlist[a].freq += 1
                found = 1
                break

        if found:
            continue
        
        new_word = Word(wdset[i])
        wordlist.append(new_word)

    return wordlist

#응집도 점수 분석 함수

def getScoredList(list):
    #heap으로 단어 길이 순으로 정렬
    heap = Heap(list)
    heap.heap_sort()
    print("heap: ", len(heap))

    score = [] #응집도,빈도를 가지는 node list
    data = heap.copy()
    
    for i in range(len(data)):
        new_node = Node(i,data[i])
        score.append(new_node)
    print('score: ',len(score))

    
    for i in range(len(score)):
        for j in range(i+1, len(score)):
            a = score[i].w.word
            b = score[j].w.word
            if type(a) == float or type(b) == float:
                print(a)
                print(b)
            if (b[:len(a)+1] == a ):
                score[j].pid = score[i].id
                score[j].dep = score[i].dep + 1
                score[i].w.freq += score[j].w.cnt
                if b == a:
                    score[j].w.word = np.nan

    
    #응집도 점수 계산
    for i in range(len(score)):
        score[i].coh = pow(score[i].w.freq/score[0].w.freq, 1/(score[i].dep+1))

    return score

In [38]:
des=data['0']

In [39]:
x = ''
for p in des:
    x += p +' '
x.replace('.','')

test_data = x.split(' ')

#함수 호출하여 점수 계산
print("words: ", len(test_data))
test_s = getWordList(test_data)
temp_s = getScoredList(test_s)

#계산 결과 데이터 프레임화
coh_score =[]
word = []
freq = []

for i in temp_s:
    coh_score.append(i.coh)
    word.append(i.w.word)
    freq.append(i.w.freq)

Coh_sentences = pd.DataFrame({'word': word,'freq':freq,'score': coh_score})
Coh_sentences.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
Coh_sentences.sort_values(by = 'score',ascending = False, inplace = True)
Coh_sentences.head(50)


words:  24590
heap:  6428
score:  6428


,word,freq,score
197,상장,800,20.000000
1997,상장을,623,17.649363
3062,마켓컬리,261,11.423660
1440,컬리는,235,10.839742
911,올해,204,10.099505
977,있다,192,9.797959
34,등,176,9.380832
1980,컬리가,175,9.354143
2738,ipo,169,9.192388
922,지난,135,8.215838


In [40]:

#정렬 후 저장
Coh_sentences.sort_values(by = 'score',ascending = False, inplace = True)
Coh_sentences.head(50)
Coh_sentences.to_csv('Kurly_Coh_words.csv', encoding = 'utf-8-sig')

In [ ]:
from konlpy.tag import Twitter
twitter=Twitter()

In [ ]:
pos_tagger=Twitter()

In [ ]:
total_news=[]
stopword = ['것','수','저']

for news in final['description'].head():
    pos_news = ['/'.join(t[:-1]) for t in pos_tagger.pos(news) if ((t[1]=='Noun') & (t[0] not in stopword))]
    total_news.append(' '.join(pos_news))

In [ ]:
total_news[1]

In [ ]:
# tf-idf 벡터 만들기
from sklearn.feature_extraction.text import CountVectorizer

kor_vectorizer = CountVectorizer(min_df=1) # 등장하는 단어들에 대한 오브젝트
kor_bow = kor_vectorizer.fit_transform(total_news) # 딕셔너리에 실제 단어들을 입력

In [ ]:
print(kor_bow)

In [ ]:
print(kor_vectorizer.get_feature_names()[:10])
kor_bow.toarray()

In [ ]:
# 생성한 TF-bow를 가지고 tf-idf 생성하기

from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer() # tfidf 변환 인스턴스 생성
res = transformer.fit_transform(kor_bow.toarray())

In [ ]:
res=tfidf.toarray()

In [ ]:
# Counter를 사용하면 단어들의 연속적인 시리즈로 된 리스트를 dictionary로 변환해줌
import collections
from collections import Counter